In [1]:
from transformers import MistralForCausalLM, LlamaForCausalLM, LongformerForMaskedLM

c:\Users\Stephan\anaconda3\envs\chatbot-qa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = MistralForCausalLM.from_pretrained("VAGOsolutions/SauerkrautLM-7b-v1-mistral")
model.config.output_attentions

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.07s/it]


MistralConfig {
  "_name_or_path": "VAGOsolutions/SauerkrautLM-7b-v1-mistral",
  "architectures": [
    "MistralForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.34.0",
  "use_cache": false,
  "vocab_size": 32000
}

In [4]:
model.model

MistralModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x MistralDecoderLayer(
      (self_attn): MistralAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): MistralRotaryEmbedding()
      )
      (mlp): MistralMLP(
        (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
        (act_fn): SiLUActivation()
      )
      (input_layernorm): MistralRMSNorm()
      (post_attention_layernorm): MistralRMSNorm()
    )
  )
  (norm): MistralRMSNorm()
)

IMPORTANT:

https://huggingface.co/docs/transformers/model_doc/mistral

# Verwendete Modelle

## Modellarchitektur LLama2 13b

Ressourcen:
- LLaMA explained ([Umar Jamil](https://www.youtube.com/watch?v=Mn_9W1nCFLo))
- LLaMA: Open and Efficient Foundation Language Models ([Hugo Touvron et al.](https://arxiv.org/abs/2302.13971))
- Llama 2: Open Foundation and Fine-Tuned Chat Models ([Hugo Touvron et al.](https://arxiv.org/abs/2307.09288))

![title](img/llama2-architecture.png)

## Modellarchitektur Mistral 7b

Ressourcen:
- Mistral 7B ([Albert Q. Jiang et al.](https://arxiv.org/abs/2310.06825))
- Mistral source code ([Huggingface](https://github.com/huggingface/transformers/blob/main/src/transformers/models/mistral/modeling_mistral.py))

![title](img/mistral-archtecture.png)

## Unterschiede Vorteile / Nachteile

Die Modellarchitekturen des LLaMA 2 und Mistral unterscheiden sich kaum.  
Die Verwendung der RMSnorm, Aufbau des Feedforward-layers, GQA (nur bei LLaMA 2 34/70b) sowie rotary positional Encoding, sind exakt gleich.  
Mistral nutzt im Gegensatz zum LLaMA 2 allerdings zusätzlich eine Sliding-Window-Attention.  
Vergleicht man die Parameter des Mistral 7b und LLaMA 2 7b, so fällt auf, dass activation function, hidden size, sowie Anzahl attention heads exakt gleich sind.  

Unterscheiden tun sich die Modelle in der Art der Attention: 
- LLaMA 2 7b: MHA
- Mistral 7b: GQA mit SWA

und in der Grösse der intermediate size, da diese des Mistral dasjenige der LLaMA 2 7/13b übersteigen.

Nach [Bau et al.](https://arxiv.org/abs/2202.05262) befinden sich die factual associations hauptsächlich in den MLP layers.  
Mistral nutzt eine grössere intermediate size als die LLaMA 2 7/13b. Möglicherweise können dadurch mehrere factual associations abgespeichert werden.  
Dies wäre eine Erklärung weshalb das Mistral in den Benchmarks im Paper von [Albert Q. Jiang et al.](https://arxiv.org/abs/2310.06825) besser performt als die genannten LLaMA 2 modelle.  

Betrachten wir die Anzahl MLP parameter so können wir folgendes beobachten:

$$
MLP\_PARAMS_{Mistral\ 7b} = 32*3*4096*14336 = 5637144576
$$

$$
MLP\_PARAMS_{LLaMA2\ 7b} = 32*3*4096*11008 = 4328521728
$$

$$
MLP\_PARAMS_{LLaMA2\ 13b} = 40*3*5120*13824 = 8493465600
$$

Das Mistral 7b besitzt mehr Parameter auf den MLPs als das LLaMA 2 7b.  
Somit kann das Mistral möglicherweise mehr Informationen aufnehmen als das LLaMA 2 7b.  
Das LLaMA 2 13b besitzt allerdings insgesamt mehr Parameter in den MLPs, weshalb man erwarten könnte, dass dies besser performen muss als das Mistral.  
Es könnte sein, dass die MLPs in den early Layers ähnlich wie bei CNNs auf kleinere Strukturen und MLPs in den late Layers auf grössere Konzepte achten.  
Dies könnte eine Erklärung dafür sein, dass das Mistral trotz weniger MLP Parameter laut den Autoren besser performt als das LLaMA 13b.  
Die bessere performance allerdings auch von den Benchmarks abhängen, da das LLaMA nicht eine solch grosse Kontext-size besitzen.  

Das Mistral hat durch die Verwendung der Sliding-window-attention definitiv einen Vorteil gegenüber dem LLaMA aufgrund der grösseren Kontext-size.  
Dadurch kann Mistral längere Texte als 4096 Tokens verarbeiten.  
Die Grouped-query-attention ist ein Tradeoff zwischen Genauigkeit und Geschwindigkeit des Modells, weshalb sowohl LLaMA als auch die Mistral diese nutzen.  

TODO: Werden die Factual associations möglicherweise besser über die MLPs verteilt durch die nutzung der SWA?

## Weshalb wurden diese Modelle verwendet

# Methodik

## Ziel

## Finetuning

### Datenset

### Training

## Evaluation

### Quantitative Analyse

### Qualitative Analyse

#### Protokoll

### Fehleranalyse

# Eda Datenset

Analyse

# Evaluationsmetriken

## Quantitative Analyse

Bleu / Rouge

## Qualitative Analyse

Factuality / Coverage

# Quantitiative Analyse

Eval loss / Bleu / Rouge

Wandb report

# Qualitative Analyse

Spaces Kontext / Frage / Anwort / Factuality / Coverage 

# Fehleranalyse / Korrekturversuch

Wo wurden Fehler gemacht?

ROME

Attention scores